<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ed73622a55e4ac822e82aedb723395e1687726632b824a0a63a00d66a72c32b6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-12 10:42:16
-------------------
qualified stocks: 84
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  5.59 L
Current:  1.44 C
-------------------
Today PnL: 98.37 K (0.69%)
Current PnL: -21.26 L (-13.93%)
CY Booked + Current PnL: -7.77 L (-5.09%)
-------------------
Total profit:  1.75 L
Total loss:  -23.01 L
-------------------
Total Booked + Current PnL: 19.21 L (15.29%)
Total Booked PnL: 40.47 L (32.22%)
Curr Year Booked PnL: 13.49 L (9.36%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.30 C
Est FTT PnL: 85.92 L (59.6%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 8.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,101.07,51.0,M-SC,2.54,86663.0,-14114.0,14204.0,-0.02,-14.01,16.39,0.09,245.0,-0.99,0.63,13.86,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.28,88620.0,-2058.0,16962.0,-0.03,-2.27,19.14,16.44,101.0,-0.12,0.64,17.45,X40,ATH,PHARMA
33,ICICIGI,2252.93,-13.14,57.0,X-MC,7.55,187590.0,13270.0,21929.0,1.05,7.61,11.69,20.19,91.0,0.61,1.35,23.88,X40,ATH,INSURANCE
41,ITC,452.00,-38.34,46.0,X-LC,0.82,198925.0,-1213.0,22101.0,-0.01,-0.61,11.11,10.44,4.0,-0.05,1.44,4.92,X40,NTT,FMCG
50,NESTLEIND,1377.00,-3.64,59.0,X-LC,8.53,296431.0,31005.0,25789.0,-0.10,11.68,8.70,21.40,11.0,1.20,2.14,19.10,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,ROUTE,2227.21,-56.49,41.0,H-SC,23.19,71321.0,-61745.0,153625.0,5.08,-46.40,215.40,69.05,118.0,-0.40,0.51,5.08,SR,ATH,IT
16,CAMS,4762.00,-2.28,58.0,X-SC,1.23,261690.0,9834.0,52600.0,3.30,3.90,20.10,24.79,122.0,0.19,1.89,27.95,X40N,NTT,MISC
38,INDUSINDBK,1800.00,294.26,76.0,L-MC,7.14,48422.0,-28784.0,54179.0,2.83,-37.28,111.89,32.89,258.0,-0.53,0.35,33.35,XR,NTT,BANKS
71,TATAELXSI,9161.00,-23.71,51.0,H-MC,6.34,103122.0,-28713.0,70938.0,2.72,-21.78,68.79,32.03,98.0,-0.40,0.74,15.51,OX40N,NTT,IT
74,TITAGARH,1548.00,-5.39,52.0,H-SC,4.18,202122.0,-40055.0,150823.0,2.59,-16.54,74.62,45.74,158.0,-0.27,1.46,31.46,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VBL,671.64,-18.08,48.0,X-LC,5.68,295011.0,-20831.0,133492.0,-1.74,-6.60,45.25,35.67,5.0,-0.16,2.13,6.35,X40N,ATH,FMCG
82,WHIRLPOOL,2270.00,-45.59,36.0,M-SC,8.82,112838.0,-1960.0,96003.0,-0.92,-1.71,85.08,81.92,223.0,-0.02,0.81,33.58,XR,NTT,DURABLES
17,CERA,9475.00,-27.21,33.0,H-SC,2.02,132710.0,-43193.0,85213.0,-0.84,-24.56,64.21,23.89,149.0,-0.51,0.96,14.61,OX40N,NTT,CERAMICS
69,SYMPHONY,1306.00,-44.94,42.0,M-SC,15.79,117317.0,-53824.0,53766.0,-0.70,-31.45,45.83,-0.03,196.0,-1.00,0.85,2.39,OX40N,NTT,DURABLES
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,37.92,147959.0,-128564.0,238643.0,-0.69,-46.49,161.29,39.81,54.0,-0.54,1.07,4.29,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WIPRO,420.0,-13.79,54.0,M-LC,5.74,151695.0,750.0,108705.0,1.23,0.5,71.66,72.51,53.0,0.01,1.09,6.55,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,101.07,51.0,M-SC,2.54,86663.0,-14114.0,14204.0,-0.02,-14.01,16.39,0.09,245.0,-0.99,0.63,13.86,OX40N,NTT,DURABLES
37,INDIGOPNTS,1408.0,126.56,76.0,M-SC,4.52,148509.0,-26050.0,26078.0,-0.41,-14.92,17.56,0.02,221.0,-1.00,1.07,29.18,OX40N,NTT,PAINTS
45,KANSAINER,340.0,-68.34,39.0,H-SC,2.28,218907.0,-50760.0,87103.0,-0.25,-18.82,39.79,13.47,138.0,-0.58,1.58,11.31,XY24,NTT,PAINTS
71,TATAELXSI,9161.0,-23.71,51.0,H-MC,6.34,103122.0,-28713.0,70938.0,2.72,-21.78,68.79,32.03,98.0,-0.40,0.74,15.51,OX40N,NTT,IT
65,SIS,528.0,2027.46,48.0,H-SC,2.21,85858.0,-25174.0,48785.0,0.81,-22.67,56.82,21.26,156.0,-0.52,0.62,16.00,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WIPRO,420.00,-13.79,54.0,M-LC,5.74,151695.0,750.0,108705.0,1.23,0.50,71.66,72.51,53.0,0.01,1.09,6.55,XR,NTT,IT
36,INDIAMART,4810.62,-52.59,60.0,H-SC,4.90,132182.0,8846.0,122784.0,1.71,7.17,92.89,106.72,119.0,0.07,0.95,32.23,AR,ATH,MISC
60,SAMMAANCAP,326.00,-193.18,56.0,M-SC,35.66,160758.0,11538.0,132641.0,1.28,7.73,82.51,96.62,208.0,0.09,1.16,73.97,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-52.59,60.0,H-SC,4.90,132182.0,8846.0,122784.0,1.71,7.17,92.89,106.72,119.0,0.07,0.95,32.23,AR,ATH,MISC
83,WIPRO,420.00,-13.79,54.0,M-LC,5.74,151695.0,750.0,108705.0,1.23,0.50,71.66,72.51,53.0,0.01,1.09,6.55,XR,NTT,IT
82,WHIRLPOOL,2270.00,-45.59,36.0,M-SC,8.82,112838.0,-1960.0,96003.0,-0.92,-1.71,85.08,81.92,223.0,-0.02,0.81,33.58,XR,NTT,DURABLES
46,KPIGREEN,731.05,2.88,44.0,H-SC,10.17,122674.0,-2623.0,61546.0,0.10,-2.09,50.17,47.03,141.0,-0.04,0.89,53.60,MH,ATH,POWER
35,IEX,219.00,-33.63,47.0,H-SC,15.86,189396.0,-10558.0,107785.0,0.16,-5.28,56.91,48.63,115.0,-0.10,1.37,5.66,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,37.92,147959.0,-128564.0,238643.0,-0.69,-46.49,161.29,39.81,54.0,-0.54,1.07,4.29,XY24,NTT,AUTO
25,GILLETTE,11206.78,-17.64,25.0,X-MC,5.73,253005.0,-891.0,83188.0,1.80,-0.35,32.88,32.42,105.0,-0.01,1.83,14.69,X40,ATH,FMCG
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-24.16,32.0,X-MC,4.71,190519.0,-9828.0,40333.0,-0.41,-4.91,21.17,15.23,178.0,-0.24,1.37,25.76,X40N,NTT,REALTY
23,DMART,5391.45,-20.25,33.0,X-LC,6.38,203150.0,-7257.0,66430.0,-0.23,-3.45,32.70,28.12,38.0,-0.11,1.47,19.36,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-38.34,46.0,X-LC,0.82,198925.0,-1213.0,22101.0,-0.01,-0.61,11.11,10.44,4.0,-0.05,1.44,4.92,X40,NTT,FMCG
52,PGHH,17907.65,-31.74,40.0,X-MC,1.09,197415.0,-3405.0,71208.0,1.88,-1.70,36.07,33.76,80.0,-0.05,1.42,3.41,X40,ATH,FMCG
64,SIEMENS,4671.50,-5.61,39.0,H-LC,1.67,151750.0,-34345.0,81824.0,0.43,-18.46,53.92,25.51,15.0,-0.42,1.10,11.83,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-8.73,38.0,X-MC,3.31,304374.0,-23617.0,130150.0,0.40,-7.20,42.76,32.48,92.0,-0.18,2.20,3.40,X40,ATH,ELECTRICAL
77,UNITDSPR,1644.00,-7.09,60.0,X-LC,3.47,243604.0,9460.0,39172.0,0.13,4.04,16.08,20.77,86.0,0.24,1.76,11.24,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-51.52,33.0,X-SC,37.13,48226.0,-16780.0,167706.0,0.79,-25.81,347.75,232.18,198.0,-0.10,0.35,0.79,XY24,NTT,MISC
1,ABB,7934.00,-42.78,33.0,H-LC,5.00,239232.0,-22387.0,141601.0,0.11,-8.56,59.19,45.57,7.0,-0.16,1.73,1.80,AR,NTT,ELECTRICAL
10,BAJAJHFL,181.50,-15.96,38.0,X-MC,6.33,177038.0,-23468.0,120616.0,0.25,-11.70,68.13,48.45,90.0,-0.19,1.28,2.09,X40N,ATH,FINANCE
20,COLPAL,3726.84,-6.08,45.0,X-MC,7.00,219020.0,-44345.0,153664.0,0.37,-16.84,70.16,41.51,84.0,-0.29,1.58,2.78,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-51.52,33.0,X-SC,37.13,48226.0,-16780.0,167706.0,0.79,-25.81,347.75,232.18,198.0,-0.10,0.35,0.79,XY24,NTT,MISC
57,RELAXO,1176.00,-39.97,46.0,X-SC,3.69,81808.0,-63352.0,139278.0,0.75,-43.64,170.25,52.31,136.0,-0.45,0.59,9.50,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.28,88620.0,-2058.0,16962.0,-0.03,-2.27,19.14,16.44,101.0,-0.12,0.64,17.45,X40,ATH,PHARMA
32,HONAUT,58357.33,-24.11,39.0,X-SC,0.93,107025.0,-20913.0,68046.0,0.48,-16.35,63.58,36.84,143.0,-0.31,0.77,9.57,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-28.04,54.0,X-LC,9.35,286418.0,-59538.0,126253.0,0.72,-17.21,44.08,19.28,1.0,-0.47,2.07,5.88,X40,ATH,IT
39,INFY,2275.00,-15.29,67.0,X-LC,4.54,330597.0,17709.0,153992.0,1.42,5.66,46.58,54.87,3.0,0.11,2.39,14.41,X40,BTT,IT
41,ITC,452.00,-38.34,46.0,X-LC,0.82,198925.0,-1213.0,22101.0,-0.01,-0.61,11.11,10.44,4.0,-0.05,1.44,4.92,X40,NTT,FMCG
80,VBL,671.64,-18.08,48.0,X-LC,5.68,295011.0,-20831.0,133492.0,-1.74,-6.60,45.25,35.67,5.0,-0.16,2.13,6.35,X40N,ATH,FMCG
1,ABB,7934.00,-42.78,33.0,H-LC,5.00,239232.0,-22387.0,141601.0,0.11,-8.56,59.19,45.57,7.0,-0.16,1.73,1.80,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,26.24,76762.0,-36787.0,76839.0,0.20,-32.40,100.10,35.27,268.0,-0.48,0.55,87.88,XR,NTT,HOTELS
7,ASIANTILES,137.00,7583.33,68.0,L-SC,15.01,84281.0,-9529.0,86144.0,0.37,-10.16,102.21,81.67,269.0,-0.11,0.61,63.21,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.60,47.0,H-SC,7.86,91200.0,-6780.0,28381.0,0.51,-6.92,31.12,22.05,152.0,-0.24,0.66,32.57,XR,ATH,FINANCE
46,KPIGREEN,731.05,2.88,44.0,H-SC,10.17,122674.0,-2623.0,61546.0,0.10,-2.09,50.17,47.03,141.0,-0.04,0.89,53.60,MH,ATH,POWER
36,INDIAMART,4810.62,-52.59,60.0,H-SC,4.90,132182.0,8846.0,122784.0,1.71,7.17,92.89,106.72,119.0,0.07,0.95,32.23,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,VAIBHAVGBL,521.00,69.70,59.0,H-SC,6.55,147063.0,-35712.0,146784.0,1.24,-19.54,99.81,60.77,125.0,-0.24,1.06,32.44,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7583.33,68.0,L-SC,15.01,84281.0,-9529.0,86144.0,0.37,-10.16,102.21,81.67,269.0,-0.11,0.61,63.21,XR,NTT,CERAMICS
37,INDIGOPNTS,1408.00,126.56,76.0,M-SC,4.52,148509.0,-26050.0,26078.0,-0.41,-14.92,17.56,0.02,221.0,-1.00,1.07,29.18,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,24.49,71.0,X-SC,6.90,220498.0,29492.0,28202.0,1.80,15.44,12.79,30.21,157.0,1.05,1.59,39.55,X40N,NTT,FINANCE
6,ASIANPAINT,3465.66,-7.57,73.0,X-LC,6.80,233288.0,-18480.0,71689.0,-0.23,-7.34,30.73,21.13,27.0,-0.26,1.68,25.48,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.52
1,25,45.97
2,50,77.54


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.08
LC    29.89
MC    25.04
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.01
X40      19.95
X40N     12.85
XY25     10.54
XR       10.02
AR        7.98
OX40N     6.81
X200      1.89
SR        1.07
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.79
X-MC    22.27
X-LC    21.61
M-SC    12.97
X-SC     5.82
H-LC     4.72
M-LC     2.53
H-MC     2.42
L-SC     1.50
L-LC     1.03
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.46,-4.06,37.72
IT,13.16,-15.20,76.87
FINANCE,12.60,-8.34,57.45
MISC,7.98,-17.48,76.73
PAINTS,5.94,-12.07,28.78
ELECTRICAL,5.80,-12.62,52.55
INSURANCE,4.62,1.28,33.50
AUTO,2.89,-43.38,103.23
BANKS,2.82,-36.96,126.22


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3112084.0,20
XR,1267514.0,13
AR,1183651.0,9
X40,928558.0,13
X40N,688169.0,9
OX40N,542003.0,9
XY25,474826.0,7
SR,272006.0,2
X200,61815.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3398865.0,24
X-MC,1322570.0,15
M-SC,1245784.0,15
X-LC,1047361.0,12
X-SC,542944.0,6
H-LC,285240.0,3
L-SC,252541.0,3
H-MC,251525.0,2
M-LC,154169.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1173000.0      6
           AR         836565.0      5
           XR         798075.0      7
M-SC       XY24       722076.0      6
X-MC       XY24       563253.0      4
X-LC       X40        427126.0      5
X-MC       X40        414320.0      7
X-LC       XY24       305462.0      2
X-SC       X40N       288126.0      4
H-SC       SR         272006.0      2
           OX40N      257673.0      3
X-LC       X40N       239094.0      3
H-LC       AR         223425.0      2
X-MC       XY25       184048.0      2
H-MC       XY24       180587.0      1
X-SC       XY24       167706.0      1
L-SC       XR         162983.0      2
X-MC       X40N       160949.0      2
M-SC       XR         143572.0      2
           XY25       132641.0      1
           OX40N      123834.0      4
           AR         123661.0      2
M-LC       XR         108705.0      1
L-SC       OX40N       89558.0      1
X-SC       X40         87112.0      1
X-LC       XY25        75679.0      2
H-MC       OX40N       70938.0      1
H-LC       X200        61815.0      1
H-SC       MH          61546.0      1
L-MC       XR          54179.0      1
M-LC       XY25        45464.0      1
L-LC       XY25        36994.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
